In [35]:
import os
import numpy as np
import csv
import torch
import sys
from datetime import datetime
from tqdm import tqdm
import pickle

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def readCSV(fname):
    file = open(f"{os.getcwd()}/{fname}.csv")
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    
    rows = []
    for row in csvreader:
        rows.append(row)
    file.close()
        
    return header, rows

In [4]:
coll_head, coll_rows = readCSV("Cleaning_collections")
api_head, api_rows = readCSV("Data_API")

In [30]:
len(api_rows)

6071027

In [5]:
api_rows[0]

['',
 '87094722.0',
 '0xed9ff1c311a611e5c457ce0a49bbbb7ed55efca988421aee27a01a8f8f6810f2',
 '0x76481caa104b5f6bccb540dae4cefaf1c398ebea',
 '',
 '0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b',
 '',
 'https://card.godsunchained.com/?id=33&q=4',
 '',
 '',
 '',
 '0.0002',
 'ETH',
 '0.030318',
 '',
 '',
 'Godsunchained',
 'Godsunchained',
 '2019-11-30 00:00:00',
 '2019-11-30 23:59:33',
 '',
 "('Godsunchained', '87094722.0')",
 'Godsunchained',
 'Games']

In [6]:
api_head

['Smart_contract',
 'ID_token',
 'Transaction_hash',
 'Seller_address',
 'Seller_username',
 'Buyer_address',
 'Buyer_username',
 'Image_url_1',
 'Image_url_2',
 'Image_url_3',
 'Image_url_4',
 'Price_Crypto',
 'Crypto',
 'Price_USD',
 'Name',
 'Description',
 'Collection',
 'Market',
 'Datetime_updated',
 'Datetime_updated_seconds',
 'Permanent_link',
 'Unique_id_collection',
 'Collection_cleaned',
 'Category']

In [22]:
el = api_rows[53]
d1 = datetime.fromisoformat(el[18]).timestamp()
d2 = datetime.fromisoformat(el[19])
coll = el[16]

In [24]:
coll

'Godsunchained'

In [31]:
data = {}

for i, rowi in tqdm(enumerate(api_rows)):
    coll = rowi[22]
    if not coll in data.keys():
        data[coll] = [i]
    else:
        data[coll] = data[coll] + [i]

2657173it [44:02, 1005.66it/s] 


KeyboardInterrupt: 

In [33]:
collections = set([row[22] for row in api_rows])

In [34]:
len(collections)

4624

In [37]:
for c in tqdm(collections):
    nfts = [row for row in api_rows if row[22] == c]
    with open(f"collections/{c}", "wb") as f:
        pickle.dump(nfts, f)

100%|███████████████████████████████████████| 4624/4624 [41:18<00:00,  1.87it/s]


# Get All Dates

In [177]:
dateranges = np.empty((len(collections),2)).astype(str)
for i, c in tqdm(enumerate(collections)):
    dateranges[i,:] = getCollectionDateRange(loadCollection(c))

with open(f"collections/stats/date-ranges", "wb") as f:
    pickle.dump(dateranges, f)

4624it [00:23, 193.88it/s] 


In [178]:
minDate = dateranges[0,1]
maxDate = dateranges[0,0]

for i in range(dateranges):
    if date[0]

array([['2021-03-02 00:51:30', '2021-03-02 00:51:30'],
       ['2021-04-20 19:56:39', '2021-04-20 19:56:39'],
       ['2021-04-26 18:07:14', '2021-03-17 01:23:41'],
       ...,
       ['2021-03-02 21:59:52', '2021-02-27 18:26:20'],
       ['2021-04-12 21:58:41', '2021-04-12 21:58:41'],
       ['2021-04-27 16:09:00', '2021-02-20 02:45:43']], dtype='<U32')

# Get All Collection Stats For Each Date

In [ ]:
# USD_mean, USD_median, USD_var, USD_sum, USD_prod
# CRYTO_mean, CRYPTO_var, CRYPTO_median, CRYPTO_sum, CRYPTO_prod, 
# CRYPTO (1=WAX), COUNT, Buyer Address, Smart Contract, Category

coll_stats = np.zeros((11))

In [162]:
def loadCollection(c):
    return pickle.load(open(f"collections/{c}", "rb"))

def getTimestamp(date):
    return datetime.fromisoformat(date).timestamp()

def getCollectionDateRange(c_nfts):
    dates = {i: getTimestamp(el[19]) for i, el in enumerate(c_nfts)}
    return c_nfts[max(dates, key=dates.get)][19], c_nfts[min(dates, key=dates.get)][19]

In [168]:
def collStats(c, start, end):
    doc_vars = [[]]*6
    start_ts = getTimestamp(start)
    end_ts = getTimestamp(end)
    
    c_nfts = loadCollection(c)
    #print(c_nfts)
    maxDate, minDate = getCollectionDateRange(c_nfts)
    
    for nft in c_nfts:
        date = nft[19]
        date_ts = getTimestamp(date)
        if date == start or date == end or (date_ts > start_ts and date_ts < end_ts):
            doc_vars[0] = doc_vars[0] + [nft[13]] # USD
            doc_vars[1] = doc_vars[1] + [nft[11]] # CRYPTO
            doc_vars[2] = doc_vars[2] + [nft[12]] # COIN
            doc_vars[3] = doc_vars[3] + [nft[5]]  # Buyer address
            doc_vars[4] = doc_vars[4] + [nft[17]] # Market
            doc_vars[5] = doc_vars[5] + [nft[23]] # Category
    
    output = []
    for el in doc_vars[0:2]:
        el = np.array(el).astype(float)
        output.append([np.mean(el), np.median(el), np.var(el), np.sum(el), np.prod(el)])
    output.append(doc_vars[2])
    output.append(doc_vars[3])
    output.append(doc_vars[4])
    output.append(doc_vars[5])
    output.append(len(c_nfts))
    return output
              

In [169]:
mn, mx = getCollectionDateRange(loadCollection(c))
print(mn)
print(mx)
stats = collStats(c, mn, mx)
print(stats)

2021-03-02 13:08:11
2021-03-02 13:08:11
[[234.13950050081365, 234.13950050081365, 0.0, 234.13950050081365, 234.13950050081365], [0.15, 0.15, 0.0, 0.15, 0.15], ['WETH'], ['0xbf202ce351d77d816b505e27535aeb22b823c469'], ['OpenSea'], ['Other'], 1]


# Create function to get stats of an nft's given collection

# Iterate Over Collections - Parse Sliding Windows

### Data to be included (for each date) for the NFT:
- All collection stats (number of items, volume traded)
- NFT collection stats (number of items, volume traded)
- NFT stats (crypto, price USD, price cypto, category)
- Label - percentage return